In [55]:
# NOTE: need to pip install ipywidgets
from IPython.display import display, clear_output
from ipywidgets import widgets
import json
def read_jsonl(file_path = 'data/7.jsonl'):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def generate_new_dicts(data, labels, user_sciper):
    new_data = []
    for review, label in zip(data, labels):
        new_data.append({'review': review['review'],'domain': review['domain'], 'label': label})
    with open(f'data/7_annotated_{user_sciper}.jsonl', 'w', encoding='utf-8') as f:
        for line in new_data:
            f.write(json.dumps(line) + '\n')

# NOTE: configure path to the data here according to your setup.
data = read_jsonl('data/7.jsonl')  
current_index = 0  
results = []  
user_sciper_global = ''
        
def show_review_and_collect_label():
    global data, current_index, results, user_sciper_global
    if current_index >= len(data):
        print("Finished all reviews!")
        generate_new_dicts(data, results, user_sciper_global)
        return
    clear_output(wait=True)
    round_show = widgets.Text(f"Round {current_index + 1}/{len(data)}", disabled=True)
    display(round_show)
    review_display = widgets.Textarea(
        value=data[current_index]['review'],
        description='Review:',
        disabled=True,
        layout={'width': 'auto', 'height': '200px'},
    )
    display(review_display)
    label_input = widgets.Text(description='Label:', placeholder='1 = positive, 0 = negative')
    display(label_input)
    submit_button = widgets.Button(description="Submit", button_style='success')
    display(submit_button)
    error_output = widgets.Output()
    display(error_output)
    def on_submit_button_clicked(b=None):
        user_input = label_input.value.strip()
        if user_input not in ['0', '1']:
            with error_output:
                clear_output(wait=True) 
                print("Error: Please enter 0 for negative or 1 for positive.")
        else:
            global current_index
            results.append(user_input)
            current_index += 1
            show_review_and_collect_label()
    submit_button.on_click(on_submit_button_clicked)
    
def fetch_sciper():
    global user_sciper_global
    input_box = widgets.Text(
        description='Input:',
        placeholder='Your SCIPER'
    )
    submit_button = widgets.Button(
        description='Submit',
        button_style='success'  
    )
    def submit_button_clicked(b=None):
        global user_sciper_global
        user_input = input_box.value.strip()
        if user_input:
            user_sciper_global = user_input
            clear_output()
            print('Your SCIPER is:', user_sciper_global)
            show_review_and_collect_label()
        else:
            clear_output()
            print('Please enter your SCIPER')
    submit_button.on_click(submit_button_clicked)
    display(input_box, submit_button)

def main():
    annotation_guide = """
    Annotation Guideline:
    
    ### Definition of positive and negative reviews: ###
    Positive review: Text expresses positive sentiment towards the product. For example, happy, satisfied, good, etc.
    Negative review: Text expresses negative sentiment towards the product. For example, disappointed, bad, poor, etc.
    
    ### Beware of sarcasms: ###
    Some reviews may first look to be positive but are actually negative. 
    For example, "I love waiting for hours for my food to arrive" is a negative review.
    
    ### Examples ###
    Positive review: "Absolutely love The Big Bang Theory DVD! Every episode is a burst of laughter and joy."
    Negative review: "Disappointed with the DVD quality of The Big Bang Theory. Episodes skip and the audio is off."
    Negative review (sarcastic): "The Big Bang Theory DVD is perfect for insomniacs. Its humor puts you to sleep faster than a lullaby!"
    """
    anno_guide_text = widgets.Textarea(
        value=annotation_guide, 
        disabled=True, 
        layout={'width': 'auto', 'height': '300px'}
    )
    display(anno_guide_text)
    confirm_button = widgets.Button(description='Start Annotation', button_style='success')
    display(confirm_button)
    
    def confirm_button_clicked(b=None):
        clear_output()
        fetch_sciper()  
    
    confirm_button.on_click(confirm_button_clicked)


main()

Textarea(value='\n    Annotation Guideline:\n    \n    ### Definition of positive and negative reviews: ###\n …

Button(button_style='success', description='Start Annotation', style=ButtonStyle())